# 数据并行
### 导入PyTorch模块并定义参数  

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [2]:
#设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### 虚拟数据集
制作一个虚拟（随机）数据集。你只需要实现getitem

In [3]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

### 简单模型
对于演示，我们的模型只获取输入，执行线性操作并提供输出。但是，您可以DataParallel在任何型号（CNN，RNN，胶囊网等）上使用  

我们在模型中放置了一个print语句来监视输入和输出张量的大小。请注意批次0级打印的内容。

In [4]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\nIn Model: input size", input.size(), "output size", output.size())

        return output

### 创建模型和DataParallel
这是本教程的核心部分。首先，我们需要创建一个模型实例并检查我们是否有多个GPU。如果我们有多个GPU，我们可以使用包装我们的模型`nn.DataParallel`。然后我们可以将我们的模型放在GPU上` model.to(device)`

In [5]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)

Let's use 2 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

### 运行模型
现在我们可以看到输入和输出张量的大小

In [6]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())


In Model: input size
In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
 torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])

In Model: input size
In Model: input size  torch.Size([15, 5]) output size torch.Size([15, 2])
torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])

In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])

In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])

In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])

In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
